In [2]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

# Precision, recall, f1

In [3]:
# 3 samples, 2 classes, 2 outputs
y_true = [[1, 0], [1, 1], [0, 1]]
y_pred = [[1, 1], [0, 0], [1, 1]]

In [4]:
metrics.precision_recall_fscore_support(y_true, y_pred, average='samples')

/usr/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.3333333333333333, 0.6666666666666666, 0.4444444444444444, None)

In [4]:
# precision = (1/2+0+1/2)/3 = 0.33
# recall = (1/1+0+1/1)/3 = 0.66

# Precision, recall and f1 @ k

In [5]:
#y_true = ['a','b','e'] = [1, 1, 0, 0, 1]
#y_pred = ['b','c','a','e','d']

# p@1: y_pred = [0, 0, 0, 0, 0]. p = 0
# p@3: y_pred = [1, 1, 1]. p = 2/3
# p@5: y_pred = [1, 1, 1, 1, 1]. p = 3/5

y_true = [[1, 1, 0, 0, 1]]
y_pred = [[1, 1, 1, 0, 0]]
metrics.precision_score(y_true, y_pred, average='samples'), metrics.recall_score(y_true, y_pred, average='samples')

(0.6666666666666666, 0.6666666666666666)

In [6]:
y_true = [[1, 1, 0, 0, 1]]
y_pred = [[1, 0, 0, 0, 0]]
metrics.precision_score(y_true, y_pred, average='samples'), metrics.recall_score(y_true, y_pred, average='samples')

(1.0, 0.3333333333333333)

In [7]:
y_true = [[0, 1, 0, 0, 1]]
y_pred = [[1, 0, 0, 0, 0]]
metrics.precision_score(y_true, y_pred, average='samples'), metrics.recall_score(y_true, y_pred, average='samples')

(0.0, 0.0)

In [8]:
def recall_at_k_score(y_true, y_pred, at_k, labels=None, average=None, sample_weight=None):
    if average not in (None, 'micro', 'macro', 'weighted', 'samples'):
        raise ValueError("{0} average is not supported".format(average))
    y_pred = select_k_best(scores=y_pred, k_best=at_k, cast_as_indicator=True)
    return metrics.recall_score(y_true, y_pred, labels=labels, average=average, sample_weight=sample_weight)

def precision_at_k_score(y_true, y_pred, at_k, labels=None, average=None, sample_weight=None): 
    if average not in (None, 'micro', 'macro', 'weighted', 'samples'):
        raise ValueError("{0} average is not supported".format(average))
    y_pred = select_k_best(scores=y_pred, k_best=at_k, cast_as_indicator=True)
    return metrics.precision_score(y_true, y_pred, labels=labels, average=average, sample_weight=sample_weight)

def select_k_best(scores, k_best, cast_as_indicator=False):
    """Select the k-best scores per sample.
    :param scores: array, shape = [n_samples, n_labels] array / sparse matrix
        Target scores, can either be probability estimates of the positive
        class, confidence values, or binary decisions.
    :param k_best: The number of best scores kept.
    :param cast_as_indicator: return multilabel-indicator matrix instead of
        matrix of scores
    :return: array/matrix of the same format that input scores
    """
    from sklearn.utils import check_array 
    from numpy import argpartition
    from scipy import sparse
    
    scores = check_array(scores, accept_sparse='csr')

    # scores_type = type_of_target(scores)
    # if scores_type not in ('continuous-multioutput', 'multilabel-indicator'):
    #     raise ValueError("{0} format is not supported".format(scores_type))

    best_scores = sparse.csr_matrix(scores, copy=True)
    for i in range(best_scores.shape[0]):
        # get the row slice per reference
        row_array = best_scores.data[
                    best_scores.indptr[i]: best_scores.indptr[i + 1]]
        # only take the k last elements in the sorted indices,
        # and set them to 0
        row_array[argpartition(row_array, kth=-k_best)[:-k_best]] = 0
    best_scores.eliminate_zeros()
    if cast_as_indicator:
        best_scores = best_scores.astype(bool).astype(int)
        if sparse.issparse(scores):
            return best_scores
        return best_scores.todense()
    if sparse.issparse(scores):
        return best_scores
    return best_scores.todense()

In [12]:
y_true = [[1, 1, 0, 1, 1, 0, 0]]
y_pred = [[0.3, 0.5, 0, 0, 0, 0, 0]] # [[0, 1, 0, 0, 0]] @1
y_pred = [[0.3, 0.5, 0, 0.9, 0, 0, 0]] # [[1, 1, 0, 0, 0]] @2
precision_at_k_score(y_true, y_pred, at_k=3, average='samples')

1.0